# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key

from api_keys import g_key

gkey = ''

gmaps.configure(api_key=gkey)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weatherfile = "weather_output_data.csv"

wdf = pd.read_csv(weatherfile)

wdf.head()


,Unnamed: 0,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1230,-134.9680,70.21,71,85,12.68,PF,1667584273
1,1,Caazapá,-26.1973,-56.3666,77.07,47,41,2.17,PY,1667584324
2,2,Bredasdorp,-34.5331,20.0428,62.71,96,51,5.35,ZA,1667584326
3,3,San Cristobal,7.7757,-72.2214,80.47,58,100,5.08,VE,1667584343
4,4,Dunedin,-45.8741,170.5036,49.80,59,31,11.99,NZ,1667584275


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
figure_layout = {
    'width': '800px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = 'HYBRID')

heatmap_layer = gmaps.heatmap_layer(
    wdf[['Latitude', 'Longitude']], weights=wdf['Humidity'],
    max_intensity=100, point_radius=20.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 95 degrees but higher than 80.
# Wind speed less than 10 mph.
# Zero cloudiness.

mask1 = ((wdf["Maximum Temperature"] > 80) & (wdf["Maximum Temperature"] < 95))
mask2 = (wdf["Wind Speed"] < 10)
mask3 = (wdf["Cloudiness"] == 0)

wdffit = wdf.loc[mask1 & mask2 & mask3].reset_index()
wdffit

,index,Unnamed: 0,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,78,78,Tessalit,20.2020,1.0125,81.19,14,0,3.80,ML,1667584924
1,267,267,Nioro Rufis,15.2305,-9.5899,89.62,20,0,4.61,ML,1667585675
2,300,300,Nouakchott,18.0792,-15.9780,94.93,24,0,8.90,MR,1667585810
3,461,461,Jeddah,21.5810,39.1654,82.31,51,0,8.05,SA,1667586448
4,465,465,Santiago Ixcuintla,21.7844,-105.2547,83.88,48,0,3.67,MX,1667586463
5,534,534,Makakilo City,21.3469,-158.0858,80.62,76,0,5.75,US,1667586645


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = wdffit.copy()
hotel_df["Hotel Name"] = ""

for i in range(len(hotel_df)):    
    # geocoordinates
    target_coordinates = str(hotel_df.loc[i,"Latitude"]) + ", " + str(hotel_df.loc[i,"Longitude"])
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    if (places_data["results"] != []):
        hotel_df.at[i,"Hotel Name"] = places_data["results"][0]["name"]


In [7]:
hotel_df

,index,Unnamed: 0,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,78,78,Tessalit,20.2020,1.0125,81.19,14,0,3.80,ML,1667584924,
1,267,267,Nioro Rufis,15.2305,-9.5899,89.62,20,0,4.61,ML,1667585675,Hotel Guetema City
2,300,300,Nouakchott,18.0792,-15.9780,94.93,24,0,8.90,MR,1667585810,Hôtel Sunset
3,461,461,Jeddah,21.5810,39.1654,82.31,51,0,8.05,SA,1667586448,Novotel Jeddah Tahlia
4,465,465,Santiago Ixcuintla,21.7844,-105.2547,83.88,48,0,3.67,MX,1667586463,Hotel Casino Plaza
5,534,534,Makakilo City,21.3469,-158.0858,80.62,76,0,5.75,US,1667586645,Hampton Inn & Suites Oahu/Kapolei


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [9]:
coordss = hotel_df.loc[:, ["Latitude", "Longitude"]]
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordss, info_box_content = hotel_info)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…